A persons writing style is an example of behaviour biometric. The words people may use and the way they strcuture their scentacnes is distinctive and can often be used to identify the author of a particular work. 

In this project I aim to compare two economists and classify who is the primary author of an academic paper based off their language. 

To ensure a large corpus, I will use Selinium to scrape Google Scholar, one of the most used academic search engines in the world. 

In [8]:
from selenium import webdriver
import requests
from tqdm import tqdm_notebook as tqdm
import time
import random

In [7]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver import ActionChains

In [1]:
import csv
import glob
import os
import sys

In [2]:
from tika import parser

In [3]:
import numpy as np
import pandas as pd

In [150]:
import textacy
import spacy
import re
import string
from spacy.lang.en import English
from spacy.lang.en.stop_words import STOP_WORDS

In [143]:
from sklearn.base import TransformerMixin

In [144]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

In [155]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression

In [157]:
from sklearn.metrics import accuracy_score
from sklearn import metrics

# Collect and download pdfs from google scholar

Main caveats of google scholar: 
- Cannot perform large scale analysis due to CAPTCHAS (robots) 
- Google scholar search enginge includes mentions of prof as well as author. 

In [8]:
def open_and_search(prof, sleep, directory):

    options = webdriver.ChromeOptions()
    options.add_experimental_option('prefs', {
    "download.default_directory": directory, #Change default directory for downloads
    "download.prompt_for_download": False, #To auto download the file
    "download.directory_upgrade": True,
    "plugins.always_open_pdf_externally": True, #It will not show PDF directly in chrome
    "plugins.extensions_to_open": "" 
    }) 
    
    driver = webdriver.Chrome('/Users/ellafranks/Downloads/chromedriver', options=options)
    driver.get('https://scholar.google.com/')
    time.sleep(sleep)

    driver.find_element_by_id('gs_hdr_tsi').send_keys(prof)

    driver.find_element_by_name('btnG').click()

    time.sleep(sleep)
    
    return driver 

In [9]:
def click_citation(driver, prof):
    time.sleep(1)
    driver.get("https://scholar.google.com/scholar?as_vis=1&q={}&hl=en&as_sdt=0,5".format(prof))
    return driver

In [10]:
def download_pdf(driver, sleep, num_pages, prof, name):
    
    for i in [i*10 for i in range(num_pages)]: 
  
        driver.get('https://scholar.google.com/scholar?start={}&q={}&hl=en&as_sdt=0,5&as_vis=1'.format(i, prof))

        time.sleep(1)

        blocks = driver.find_elements_by_class_name('gs_r.gs_or.gs_scl')

        for block in blocks:
            try: 
                if block.find_element_by_class_name('gs_or_ggsm'):
                    if name in block.find_element_by_class_name('gs_a').text:
                        block.find_element_by_class_name('gs_ctg2').click()
            except:
                pass


**This next line of code is made up of 3 functions:**

1. Navigate to google scholar page, search up associated prof name
2. Apply filters ensuring prof is a key author and not just a citation
3. Download papers that are pdf formatted AND associated prof name as primary author into chosen directory 

These two prof were chosen due to them having written papers with similar content, but never coauthoring a paper together.

In [11]:
directory1 = "/Users/ellafranks/Desktop/Other-projects/Guess_the_prof/Julian_Franks_papers/"
directory2 = "/Users/ellafranks/Desktop/Other-projects/Guess_the_prof/La_Porta_papers/"

In [13]:
#Prof 1 - Julian Franks
download_pdf(click_citation(open_and_search('Julian Franks', 1, directory1), 'Julian+Franks'), 1, 20, 'Julian+Franks', 'Franks')

In [410]:
#Prof 2 - Rafael La Porta
download_pdf(click_citation(open_and_search('La Porta', 1, directory2), 'La+Porta'), 1, 20, 'La+Porta', 'Porta')

# Parse and convert pdf to raw text

Using Tika to extract content from pdfs (need to install the latest version of Java in order to run the script - Can use pdfMiner3 or pdfPlumber if not)

In [22]:
def grab_pdf_content(filepath):
    
    list_pdf, input_files = [], []
    
    #grab all PDF's file name and strip out .pdf
    all_files = glob.glob(os.path.join(filepath, "*.pdf"))
    for input_file in all_files:
        filename = os.path.basename(input_file)
        filename = filename.strip('.pdf')
        
    # Use Tika to parse the PDF's, extract text content
        parsedPDF = parser.from_file(input_file)
        pdf = parsedPDF["content"]
        
        list_pdf.append(pdf)
        input_files.append(filename)
 
    return input_files, list_pdf

In [23]:
def convert_to_df(n, c): 
    corpus = list(zip(n, c))
    df = pd.DataFrame.from_dict({name:doc for name,doc in corpus}, orient='index', columns=['text'])
    df.to_csv('{}_pdfs_df.csv'.format(df.shape[0]))

In [411]:
filepath = "/../Users/ellafranks/Desktop/Other-projects/Guess_the_prof/Julian_Franks_papers"
filepath2 = "/../Users/ellafranks/Desktop/Other-projects/Guess_the_prof/La_Porta_papers"

1. Extract filename and content from Julian Franks
2. Compile and save pdfs into a csv dataframe 
3. Add relevent column (prof) and combine dataframes

In [197]:
# Julian Franks
name, content = grab_pdf_content(filepath)
convert_to_df(name, content)

In [412]:
# La Porta
name2, content2 = grab_pdf_content(filepath2)
convert_to_df(name2, content2)

In [5]:
JF = pd.read_csv('38_pdfs_df.csv', sep=',', index_col=[0])
LP = pd.read_csv('59_pdfs_df.csv', sep=',', index_col=[0])

In [6]:
JF['prof'] = 'Julian'
LP['prof'] = 'Rafael'

In [102]:
comb = pd.concat([JF, LP])
comb.shape

(97, 2)

# Clean and prepare text

MORE RULES?

- REMOVE REFERENCES?
- REMOVE FIGURES?

In [103]:
def general_clean(content):
    paragraphs = str(content).split('\n\n')
    list_stripped = []  
    for item in paragraphs:
        stripped = item.strip().replace(',', '').lower()
        list_stripped.append(stripped)
    
    #remove if empty cell or paragraph break 
    final_list = []
    for l in list_stripped:
        if l != '' and '\n' not in l:
            final_list.append(l)  
    return final_list

Above I split the content by newline paragraphs, strip them and convert to lowercase and then remove the cell if empty or contains a paragraph break. I then 'explode' the list of lists per row into many rows.

In [104]:
comb.text = comb.text.apply(lambda x: general_clean(x))

In [105]:
df = comb.explode('text')

In [108]:
print('There are {} duplictates found in {} rows.'.format(df.duplicated().sum(), df.shape[0]))

There are 10001 duplictates found in 49564 rows.


I have a very large corpus and in order to make this clean as applicable to all pdfs and their authors, I drop duplicate cells which mostly contain page numbers and chapter references. 

In [109]:
df.drop_duplicates(inplace=True)

In [110]:
print('I will drop {} rows which are less than 10 words long.'.format(df.text.apply(lambda x: len(x.split(' ')) < 10).sum()))

I will drop 13438 rows which are less than 10 words long.


In [111]:
df = df[df['text'].apply(lambda x: len(x.split(' ')) > 10)]

In [112]:
df = df.reset_index().rename(columns={'index': 'paper'})

Remove all occurances of prof name and all instances where cells begin with an asterix as they tend to be footnotes 

In [113]:
prof = 'Julian|Franks|Rafael|Porta|La'

print('There are {} number of instances of the prof name, so we shall replace these.'.\
      format(df[df['text'].str.contains(prof, flags=re.IGNORECASE)].shape[0]))

There are 6924 number of instances of the prof name, so we shall replace these.


In [119]:
prof = ['julian', 'franks', 'rafael', 'porta', 'la']
df.text = df.text.replace({l: '' for l in prof}, regex=True)

In [139]:
df = df[df.text.map(lambda x: not (x.startswith('*')))]

We would ideally like the classes as balenced as possible

In [141]:
df.groupby('prof')[['text']].count()

,text
prof,
Julian,15311
Rafael,9166


# Process text using spacy and make prediction

**Build a classification model to predict if the economics paper was Julian Franks or La Porta**

In [166]:
df['prof'].value_counts(normalize=True).max()

0.6255260040037587

- ngrams, max features
- remove dots, punctuation, 

- **Creating our tokenizer function that returns a list of preproccessed tokens that are lemmatised, lowercased and have had stopwords removed.**

- **Custom transformer using spaCy**

In [167]:
punctuations = string.punctuation
stop_words = STOP_WORDS
parser = English()

def spacy_tokenizer(sentence):
    mytokens = parser(sentence)
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]
    return mytokens

class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        # Cleaning Text
        return [clean_text(text) for text in X]

    def fit(self, X, y=None, **fit_params):
        return self

    def get_params(self, deep=True):
        return {}

# Basic function to clean the text
def clean_text(text):
    # Removing spaces and converting text into lowercase
    return text.strip().lower()

In [213]:
X = df['text']
y = df['prof'] 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, shuffle=True, stratify=y)

In [214]:
tfidf_vector = TfidfVectorizer(tokenizer = spacy_tokenizer)

classifier = LogisticRegression(solver = 'lbfgs')

pipe = Pipeline([("cleaner", predictors()),
                 ('vectorizer', tfidf_vector),
                 ('classifier', classifier)])

pipe.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('cleaner', <__main__.predictors object at 0x1a2d905dd8>), ('vectorizer', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ng...enalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False))])

In [215]:
predicted = pipe.predict(X_test)
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))

Logistic Regression Accuracy: 0.9107434640522876


In [217]:
importances = pd.DataFrame({'feature': pipe.named_steps['vectorizer'].get_feature_names(),
                        'coef': pipe.named_steps['classifier'].coef_[0],
                        'abs_coef': np.abs(pipe.named_steps['classifier'].coef_[0])}).sort_values('coef', ascending=False)
importances

,feature,coef,abs_coef
8028,government,6.472846,6.472846
10668,nodes,4.158441,4.158441
10342,mobile,4.044718,4.044718
9536,legal,4.040153,4.040153
8436,human,3.995545,3.995545
...,...,...,...
4169,bankruptcy,-4.337710,4.337710
14637,uk,-4.655064,4.655064
14605,u.k,-4.893694,4.893694
5266,companies,-5.264934,5.264934


In [218]:
STOPLIST = set(stopwords.words('english') + list(ENGLISH_STOP_WORDS))
SYMBOLS = " ".join(string.punctuation).split(" ") + ["-", "...", "”", "”"]
class CleanTextTransformer(TransformerMixin):
    def transform(self, X, **transform_params):
        return [cleanText(text) for text in X]
    def fit(self, X, y=None, **fit_params):
        return self
def get_params(self, deep=True):
        return {}
    
def cleanText(text):
    text = text.strip().replace("\n", " ").replace("\r", " ")
    text = text.lower()
    return text
def tokenizeText(sample):
    tokens = parser(sample)
    lemmas = []
    for tok in tokens:
        lemmas.append(tok.lemma_.lower().strip() if tok.lemma_ != "-PRON-" else tok.lower_)
    tokens = lemmas
    tokens = [tok for tok in tokens if tok not in STOPLIST]
    tokens = [tok for tok in tokens if tok not in SYMBOLS]
    return tokens

In [219]:
train, test = train_test_split(df, test_size=0.2, shuffle=True, random_state=42)

In [225]:
def printNMostInformative(vectorizer, clf, N):
    feature_names = vectorizer.get_feature_names()
    coefs_with_fns = sorted(zip(clf.coef_[0], feature_names))
    topClass1 = coefs_with_fns[:N]
    topClass2 = coefs_with_fns[:-(N + 1):-1]
    print("Class 1 best: ")
    for feat in topClass1:
        print(feat)
    print("Class 2 best: ")
    for feat in topClass2:
        print(feat)
        
vectorizer = TfidfVectorizer(tokenizer=tokenizeText, ngram_range=(1,1))
clf = LinearSVC()

pipe = Pipeline([('cleanText', CleanTextTransformer()), ('vectorizer', vectorizer), ('clf', clf)])
# data
train1 = train['text'].tolist()
labelsTrain1 = train['prof'].tolist()
test1 = test['text'].tolist()
labelsTest1 = test['prof'].tolist()

In [226]:
# train
pipe.fit(train1, labelsTrain1)
# test
preds = pipe.predict(test1)
print("accuracy:", accuracy_score(labelsTest1, preds))
print("Top 10 features used to predict: ")

printNMostInformative(vectorizer, clf, 20)
pipe = Pipeline([('cleanText', CleanTextTransformer()), ('vectorizer', vectorizer)])
transform = pipe.fit_transform(train1, labelsTrain1)
vocab = vectorizer.get_feature_names()
for i in range(len(train1)):
    s = ""
    indexIntoVocab = transform.indices[transform.indptr[i]:transform.indptr[i+1]]
    numOccurences = transform.data[transform.indptr[i]:transform.indptr[i+1]]
    for idx, num in zip(indexIntoVocab, numOccurences):
        s += str((vocab[idx], num))

accuracy: 0.9268790849673203
Top 10 features used to predict: 
Class 1 best: 
(-2.648986744274183, 'uk')
(-2.3825863401634937, 'u.k')
(-2.2774710313825546, 'companies')
(-2.1908991429411535, 'bankruptcy')
(-2.147959857170371, 'family')
(-2.106737782657793, 'repurchases')
(-1.9652703665389022, 'innovation')
(-1.9470774719221438, 'bias')
(-1.9464173180338016, 'engagement')
(-1.8067363155920442, 'acquisitions')
(-1.7878129624404822, 'turnover')
(-1.78555150325406, 'activism')
(-1.7607516085203698, 'lenders')
(-1.7201163894887195, 'diverged')
(-1.7147090266210705, 'outsider')
(-1.70986213512913, '1900')
(-1.685331902608087, 'liquidation')
(-1.6779837263859279, 'active')
(-1.6616356304353777, 'fund')
(-1.65405984315486, 'chapter')
Class 2 best: 
(3.0572213109157027, '')
(2.944518022800794, 'government')
(2.7561543408867952, 'schooling')
(2.5719590912360517, 'bor')
(2.520709738660581, 'human')
(2.4536669839294225, 'expropriation')
(2.370010575927269, 'key')
(2.33555475088195, 'protocol')
(2.